In [ ]:
from sodapy import Socrata
import cartopy.crs as ccrs
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from getpass import getpass
from datetime import datetime, timedelta

Area = ['Forestry', 'Landscape', 'Traffic', 'Streets', 'Stormwater',
       'Street Sweeping', 'Bridges', 'Environmental', 'Streets_Bridges',
       'Wastewater', 'Miscellaneous', 'Water Distribution',
       'Special Events']
cats = pd.read_csv("categories.csv").to_dict()
civic_leagues = pd.read_csv("civic_leagues.csv")


In [ ]:

apptoken = widgets.Text()
area = widgets.Dropdown(options=Area)
civicleague = widgets.Dropdown(options=civic_leagues['0'].values)
startdate = widgets.NaiveDatetimePicker(value = datetime.today())
enddate = widgets.NaiveDatetimePicker(value = startdate.value + timedelta(days=7))
search = widgets.Button(description="Search")
out = widgets.Output()

search.on_click(lambda x: get_work_orders(startdate, enddate))

display(widgets.HBox([widgets.Label("Enter App Token:"), apptoken]), widgets.HBox([area, civicleague, startdate, enddate, search]), out)

def get_work_orders(startdate, enddate):
    out.clear_output()
    work_orders = "qzfe-wj25"
    client = Socrata("data.norfolk.gov", app_token=apptoken.value)

    query = f"start_date between '{startdate.value.isoformat()}' and '{enddate.value.isoformat()}'"
    if civicleague.value != "":
        query += f" and civic_league='{civicleague.value}'"
    
    if area.value != "":
        query +=  f" and area='{area.value}'"

    try:
        items = pd.DataFrame(client.get(work_orders, where=query))

        out.append_display_data(items)
    except Exception as ex:
        out.append_display_data(ex)
    
    client.close()


Output()